In [15]:
#没有replicated文档，多个EPS
#统计每条chain进入pore的次数

import pandas as pd
import numpy as np
import math

In [16]:

#1_rearrange
#1_add chain number, time step

#load file
#EPS = ['0.20', '0.40', '0.60', '0.80', '1.00', '1.20', '1.40', '1.60', '1.80', '2.00']
EPS = ['0.20']

for z in EPS:
    fread1 = open('C:\\Users\\jhyan\\Documents\\all\\projects\\csml_python_research_code\\py_count_chains_in_pore\\short.lammpstrj', 'r')
    longread1 = fread1.readlines()
        
    #strip, split and save data
    longread2 = []
    for j in longread1:
        first = j.strip('/n')
        second = first.split()
        longread2.append(second)

    #delete title in dump file
    longdump = []
    linelength = 8 #the length of the line that should be used for coding in dump file*************
    for j in range(0, len(longread2), 1):
        if len(longread2[j]) == linelength:
            longdump.append(longread2[j])

    #add time step
    timesteplong = []
    idumpstep = 101000000 # timestep the dump start*************************
    dumpsegment = 100000  # dump every time step****************************
    totalmononumberlong = 1300  # atoms number******************************
    for j in range(1, len(longdump) // totalmononumberlong + 1, 1):
        for k in range(1, totalmononumberlong + 1, 1):
            timesteplong.append(idumpstep)
        idumpstep = idumpstep + dumpsegment
        
    #add chain number
    longchainnumber = []
    longchainlength = 20 #chain length************************
    for j in range(1, len(longdump)//totalmononumberlong+1, 1):
        p = 0
        for k in range(1, totalmononumberlong//longchainlength+1, 1):
            p = p + 1
            for h in range(1, longchainlength+1, 1):
                longchainnumber.append(p)
        #monomer 总数和链长不能整除的时候
        for h in range(1, totalmononumberlong-totalmononumberlong//longchainlength*longchainlength+1, 1):
            longchainnumber.append(totalmononumberlong//longchainlength+1)

    #write dump into a new file
    fwrite1 = open('long_rearrange.txt', 'w')
    a = 'ID TYPE x y z ix iy iz ' + '\n'
    fwrite1.write(a)
    for j in range(0, len(longdump), 1):
        c = str(longdump[j]).replace('[','').replace(']','')
        c = c.replace("'",'').replace(',','') + '\n'
        fwrite1.write(c)
    fwrite1.close()

    #write timestep into a new file
    fwrite1 = open('long_time.txt', 'w')
    a = 'timestep' + '\n'
    fwrite1.write(a)
    for j in range(0, len(timesteplong), 1):
        a = str(timesteplong[j]).replace('[', '').replace(']', '')
        a = a.replace("'", '').replace(',', '') + '\n'
        fwrite1.write(a)
    fwrite1.close()
                
    #write chain number into a new file
    fwrite1 = open('long_chainnumber.txt', 'w')
    a = 'chainnumber' + '\n'
    fwrite1.write(a)
    for j in range(0, len(longchainnumber), 1):
        a = str(longchainnumber[j]).replace('[', '').replace(']', '')
        a = a.replace("'", '').replace(',', '') + '\n'
        fwrite1.write(a)
    fwrite1.close()


#2_pandas: particles in pore
    #load files
    dfl = pd.read_table('long_rearrange.txt', delim_whitespace=True)
    dfl_t = pd.read_table('long_time.txt', delim_whitespace=True)
    dfl_c = pd.read_table('long_chainnumber.txt', delim_whitespace=True)
    #calculate real coordination
    xbox = 30 #the total length of simulation box*******************
    ybox = 30 #the total length of simulation box*******************
    zbox = 61 #the total length of simulation box*******************
    dfl['realx'] = dfl.x + xbox*dfl.ix
    dfl['realy'] = dfl.y + ybox*dfl.iy
    dfl['realz'] = dfl.z + zbox*dfl.iz
        
    #add timestep
    dfl['timestep'] = dfl_t['timestep']
    
    #add chainnumber
    dfl['chainnumber'] = dfl_c['chainnumber']
        
    #select particles inside pore
    bottom_side = -31 #下半截tube的长度***************
    rc = 2.5 #截断半径********************************
    bottom_wall = bottom_side + rc
    
    dfl_contact_z = dfl.loc[(dfl['realz'] <= 0)]

    #write files
    dfl_contact_z.to_csv('particles_inside_pore.txt', sep=' ', header=None, index=None)

    
#3_chain_inside_pore
    #统计每条chain进入pore的次数
    #load file
    freadinital1 = open('particles_inside_pore.txt', 'r')
    initial = freadinital1.readlines()

    #strip and split data
    long = []
    for j in initial:
        first = j.strip('/n')
        second = first.split()
        long.append(second)

    #统计每条chain进入pore的次数
    #创建chain number
    if int(totalmononumberlong%longchainlength) == int(0):
        totalchainnumber = totalmononumberlong//longchainlength #number of polymer chain***********************
        chainnumber_np = np.linspace(1, totalchainnumber, totalchainnumber)
        chainnumber = []
        for i in chainnumber_np:
            chainnumber.append(int(i))
    else:
        totalchainnumber = totalmononumberlong//longchainlength+1 #number of polymer chain***********************
        chainnumber_np = np.linspace(1, totalchainnumber, totalchainnumber)
        chainnumber = []
        for i in chainnumber_np:
            chainnumber.append(int(i))
   
    #统计chain在pore的次数
    number = []
    number_re = []
    for i in range(0, len(chainnumber), 1):
        for j in range(0, len(long), 1):
            if int(long[j][12]) == int(chainnumber[i]):
                number.append(int(long[j][12]))            
        if len(number) == 0:
            number_re.append(0)
            number = []
        else:
            number_re.append(len(number))
            number = []
    
    #write into a new file
    fwrite1 = open('long_number_in_pore.txt', 'w')
    for j in range(0, len(chainnumber), 1):
        s = str(chainnumber[j]).replace('[','').replace(']','')
        s = s.replace("'",'').replace(',','') + ' '
        d = str(number_re[j]).replace('[','').replace(']','')
        d = d.replace("'",'').replace(',','') +'\n'
        fwrite1.write(s)
        fwrite1.write(d)
    fwrite1.close()
